# Global import


In [1]:
from pathlib import Path

from mistralai import Mistral

from app.proposal_object.schemas import Devis

from app.proposal_object.path import ProposalObjectPaths

from app.proposal_object.proposal_object import ProposalObjectHandler


/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(row[0], str) and row[0].strip() == "Lot :":
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  isinstance(row[2], str)
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

# Proposals path


In [2]:
### Hard ###
FILE_NAME = "Devis DE2502-580 - LABOURDETTE - "


### Medium ###
# FILE_NAME = "Lot Electricité - JUILLARD - Devis_n°OF9206"

# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1"

# FILE_NAME = "Devis_'OF9189'"

# FILE_NAME = "Devis DE2412-461 - GOURDON - "

### Easy ###
# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - Vouland   Seignosse"

proposal_path = ProposalObjectPaths(file_name=FILE_NAME)


# Proposal Object


# Mistral Client


In [3]:
mistral_api_key = "hSRyFLyIbO0kzD7PR948CtyhLpjUQWBY"

model = "mistral-large-2411"

client = Mistral(api_key=mistral_api_key)


# Upload doc


In [4]:
uploaded_pdf = client.files.upload(
    file={
        "file_name": FILE_NAME,
        "content": open(proposal_path.proposal_path, "rb"),
    },
    purpose="ocr",
)

signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

print(f"Signed URL: {signed_url.url}")


Signed URL: https://mistralaifilesapiprodswe.blob.core.windows.net/fine-tune/5fda61c1-c371-40c7-906b-72b14c1b9202/b2f4f412-dd41-4f47-a726-1f050aed58d8/979afdc14c84434d954f41028edc0492.pdf?se=2025-06-13T11%3A41%3A08Z&sp=r&sv=2025-05-05&sr=b&sig=Gt/LeHqA/12DrFESQ7GPpmZMXw6FCBAHG81kwXvXJq4%3D


# LLM + Vision call


In [5]:
PROPOSAL_ANALYZER_SYSTEM_PROMPT = """Vous êtes le meilleur analyseur de devis du monde. Vous devez reconstruire le devis à partir de l'OCR. Utilisez les polygones pour trouver la place des éléments. Pour chaque produit du devis, vous devez très précisément noter son label, sa description, le lot auquel il appartient son prix HT la quantité et la TVA. Le produits peuvent être grouper par catégorie et ou contenir des sous catégories de produits."""


In [6]:
messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": PROPOSAL_ANALYZER_SYSTEM_PROMPT},
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "document_url", "document_url": signed_url.url},
        ],
    },
]

chat_response = client.chat.parse(
    model=model, messages=messages, temperature=0, response_format=Devis
)

proposal_structured: Devis = Devis.model_validate_json(
    chat_response.choices[0].message.content
)

proposal_json = proposal_structured.model_dump_json(indent=2)

a_out = Path(proposal_path.proposal_object_predicted_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(proposal_json)

print(f"✅ PROPOSAL STRUCTURED write in {a_out.resolve()}")


✅ PROPOSAL STRUCTURED write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_predicted/Devis DE2502-580 - LABOURDETTE - .json


# Proposal object


In [7]:
ProposalObjectHandler(proposal_path=proposal_path).analyze_proposal_object()


2025-06-12 13:49:56 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_text.49]:                             ✅ PROPOSAL TEXT write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_text/Devis DE2502-580 - LABOURDETTE - .txt


2025-06-12 13:49:56 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_pdf.65]:                              ✅ PROPOSAL PDF write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_pdf_object/Devis DE2502-580 - LABOURDETTE - .pdf
2025-06-12 13:49:56 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_proposal_lines.94]:                   ✅ PROPOSAL LINES write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_line/Devis DE2502-580 - LABOURDETTE - .txt
2025-06-12 13:49:56 INFO -> [app/proposal_object/proposal_object.py.proposal_object.validate_proposal_object.116]:                   {
  "computed_total_ht": 10707.38,
  "computed_total_tva": 1070.02,
  "computed_total_ttc": 11777.4,
  "computed_total_cout_additionnel": 0.0,
  "logs": [
    "📑 Démarrage de la vérification des prix pour le devis :",
    "📋 Feuille « Préparation » : HT = 135.00, TVA = 13.50",
    "📋 Feuille « Dépose de la chaudière et de 13 radiateurs et mise en

arraybcimunnm = [
    "|    PACK   |   TVA   |                                                                                      LABEL                                                                                      |                                                                                   DESCRIPTION                                                                                   | QTY | UNIT | UNIT_PRICE_HT | TOTAL_HT |",
    "|-----------|---------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----|------|---------------|----------|",
    "| SANITAIRE | TVA 10% |                                                                                   Préparation      